In [179]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import re
from dateutil import parser


batchA = pd.read_csv('batchA.csv').T
batchB = pd.read_csv('batchB.csv').T
batchA['batch'] = '1'
batchB['batch'] = '2'
allBatch = pd.read_csv('allBatch.csv', index_col=0).T
allBatch.reset_index(drop=False, inplace=True)
allBatch = allBatch.rename_axis(None, axis=1)
allBatch = allBatch.rename(columns={'index': 'Pre-wetting date'})

# cleaning columns
dateColumns = allBatch.columns[0:2]
percentColumns = allBatch.columns[[4, 5, 8, 11, 12]]
numericalCols = allBatch.columns[2:18]
allBatch[percentColumns] = allBatch[percentColumns].apply(lambda x : x.str.strip('%').astype(float)/100)
allBatch[numericalCols] = allBatch[numericalCols].applymap(lambda x: re.sub("[^0-9]", "", x) if type(x) == str else x)
allBatch[numericalCols] = allBatch[numericalCols].apply(pd.to_numeric, errors='coerce', downcast='float')


TypeError: Parser must be a string or character stream, not Series

In [177]:
allBatch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147 entries, 0 to 146
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Pre-wetting date              147 non-null    object 
 1   Evacuation date (B)           136 non-null    object 
 2   Wet straw at lay-out (T)      143 non-null    float32
 3   Dry straw at lay-out (T)      143 non-null    float32
 4   Moisture content PII          142 non-null    float32
 5   Conversion rate               142 non-null    float32
 6   Fill bag rate                 142 non-null    float32
 7   Spawn rate (ml)               142 non-null    float32
 8   Compost score                 112 non-null    float32
 9   # bags (WHITE)                142 non-null    float32
 10  # bags (BROWN)                130 non-null    float32
 11  Yield/dry straw (kg) (WHITE)  138 non-null    float32
 12  Yield/dry straw (kg) (BROWN)  138 non-null    float32
 13  Yield